# Machine translation using RNN

![](https://pytorch.org/tutorials/_images/seq2seq.png)

In [3]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata

import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cuda')

In [7]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {} #
        self.word2count = {} # words counter
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        # Add words splitting the sentence by space
        ### BEGIN SOLUTION
        for word in sentence.split(' '):
            self.addWord(word)
        ### END SOLUTION

    def addWord(self, word):
        # Update self.word2index, self.word2count, self.n_words
        ### BEGIN SOLUTION
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
        ### END SOLUTION

In [12]:
#lang = Lang("eng")
lang.addSentence("could you please stop the noise")
lang.word2count, lang.word2index, lang.index2word, lang.n_words

({'bye': 4,
  'my': 1,
  'dear': 1,
  'could': 1,
  'you': 1,
  'please': 1,
  'stop': 1,
  'the': 1,
  'noise': 1},
 {'bye': 2,
  'my': 3,
  'dear': 4,
  'could': 5,
  'you': 6,
  'please': 7,
  'stop': 8,
  'the': 9,
  'noise': 10},
 {0: 'SOS',
  1: 'EOS',
  2: 'bye',
  3: 'my',
  4: 'dear',
  5: 'could',
  6: 'you',
  7: 'please',
  8: 'stop',
  9: 'the',
  10: 'noise'},
 11)

In [13]:
test_lang = Lang('test')
test_lang.addSentence('Hello, World!')
assert test_lang.n_words == 4
assert test_lang.word2index == {'Hello,': 2, 'World!': 3}
assert test_lang.word2count == {'Hello,': 1, 'World!': 1}

In [14]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [15]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [16]:
!wget https://download.pytorch.org/tutorial/data.zip
!unzip data.zip

--2024-08-21 07:58:30--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 18.239.83.126, 18.239.83.16, 18.239.83.32, ...
Connecting to download.pytorch.org (download.pytorch.org)|18.239.83.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   2.75M  --.-KB/s    in 0.03s   

2024-08-21 07:58:30 (85.4 MB/s) - ‘data.zip’ saved [2882130/2882130]

Archive:  data.zip
   creating: data/
  inflating: data/eng-fra.txt        
   creating: data/names/
  inflating: data/names/Arabic.txt   
  inflating: data/names/Chinese.txt  
  inflating: data/names/Czech.txt    
  inflating: data/names/Dutch.txt    
  inflating: data/names/English.txt  
  inflating: data/names/French.txt   
  inflating: data/names/German.txt   
  inflating: data/names/Greek.txt    
  inflating: data/names/Irish.txt    
  inflating: data/names/Ital

In [18]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [38]:
sentence = "i am a bit scared of everything"
sentence.startswith(eng_prefixes), len(sentence.split(" "))
pair = [sentence, "Sono spaventato"]
filterPair(pair)
pair = ["Sono spaventato", sentence]
filterPair(pair)

True

In [24]:
filterPair(["I'm", "I am"])

False

In [40]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

# Uncomment if data does not exists
# !wget https://download.pytorch.org/tutorial/data.zip
# !unzip data.zip > /dev/null

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['il embrasse tres bien .', 'he s a great kisser .']


In [41]:
def indexesFromSentence(lang, sentence):
    # split sentence by space and convert words to indices
    ### BEGIN SOLUTION
    return [lang.word2index[word] for word in sentence.split(' ')]
    ### END SOLUTION

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [49]:
indexesFromSentence(output_lang, "i will try to fix you")

[2, 1700, 1097, 532, 2219, 129]

In [56]:
ptt = tensorFromSentence(output_lang, "i will try to fix you")
print (ptt)
print (ptt.shape), print (ptt.dtype)

tensor([[   2],
        [1700],
        [1097],
        [ 532],
        [2219],
        [ 129],
        [   1]], device='cuda:0')
torch.Size([7, 1])
torch.int64


(None, None)

In [55]:
assert indexesFromSentence(output_lang, 'master driver') == [975, 977]

In [62]:
nn.GRU?

In [63]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size) # Special learnable layer which converts tensor of word indices to tensor of hidden size
        # Define your RNN here
        ### BEGIN SOLUTION
        self.gru = nn.GRU(hidden_size, hidden_size)
        ### END SOLUTION

    def forward(self, input, hidden):
        # HINT 1: input is single sentence of word indexes
        # HINT 2: PyTorch RNNs takes [seq_len, batch_size, hidden_size] tensor as input
        ### BEGIN SOLUTION
        embedded = self.embedding(input).view(-1, 1, self.hidden_size)
        output = embedded
        output, hidden = self.gru(output, hidden)
        ### END SOLUTION
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [74]:
nn.Embedding?

In [64]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        ### BEGIN SOLUTION
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        ### END SOLUTION

    def forward(self, input, hidden):
        ### BEGIN SOLUTION
        output = self.embedding(input).view(-1, 1, self.hidden_size)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        ### END SOLUTION
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [160]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH,teacher_forcing_ratio=0.5):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(
            decoder_input, decoder_hidden)
        if use_teacher_forcing: # Teacher forcing: Feed the target as the next decoder input
            ### BEGIN SOLUTION
            decoder_input = target_tensor[di]  # Teacher forcing
            ### END SOLUTION
        else:                   # Without teacher forcing: use its own predictions as the next input
            ### BEGIN SOLUTION
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as decoder input
            ### END SOLUTION
        loss += criterion(decoder_output, target_tensor[di])
        # Stop if terminate token (EOS) is generated
        ### BEGIN SOLUTION
        if decoder_input.item() == EOS_token: # Terminate token is returned. Stop
            break
        ### END SOLUTION

    # Perform gradient step
    ### BEGIN SOLUTION
    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()
    ### END SOLUTION

    return loss.item() / target_length

In [153]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [154]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)


In [155]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [156]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()


        decoded_words = []
        # Translate the sentence. Substitute EOS_token with "<EOS>" and append to the end of decoded_words

        ### BEGIN SOLUTION
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        ### END SOLUTION
        return decoded_words

In [162]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [157]:
hidden_size = 256
encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, output_lang.n_words).to(device)

In [158]:
input_lang.n_words, output_lang.n_words

(4345, 2803)

In [87]:
pair = random.choice(pairs)
training_pair = tensorsFromPair(pair)
pair, training_pair

(['je suis un peu occupe .', 'i m a little busy .'],
 (tensor([[  6],
          [ 11],
          [ 66],
          [635],
          [ 30],
          [  5],
          [  1]], device='cuda:0'),
  tensor([[  2],
          [  3],
          [ 42],
          [884],
          [ 19],
          [  4],
          [  1]], device='cuda:0')))

In [110]:
pairs[40][1]

'i m safe .'

In [111]:
lang = [input_lang, output_lang]
for j in range(2):
  print ("-----")
  for i in range(training_pair[j].size(0)):
    print(lang[j].index2word[int(training_pair[j][i][0])])

-----
je
suis
un
peu
occupe
.
EOS
-----
i
m
a
little
busy
.
EOS


In [ ]:
for i in range(training_pair[1].size(0)):
  print(input_lang.index2word[int(training_pair[0][i][0])])

In [85]:
input_tensor, output_tensor = training_pair[0], training_pair[1]

In [86]:
input_length, target_length = input_tensor.size(0), output_tensor.size(0)
input_length, target_length

(7, 6)

In [115]:
encoder_hidden = encoder.initHidden()

In [117]:
encoder_hidden, encoder_hidden.shape

(tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [118]:
encoder_output

NameError: name 'encoder_output' is not defined

In [119]:
encoder_output, encoder_hidden = encoder(input_tensor[0], encoder_hidden)

In [121]:
encoder_output.shape, encoder_output

(torch.Size([1, 1, 256]),
 tensor([[[-0.3660, -0.1374, -0.1627,  0.1791,  0.1956,  0.0855, -0.2766,
            0.0155, -0.1747,  0.1660, -0.0397, -0.0998,  0.1683,  0.0571,
            0.4571, -0.1024,  0.1151, -0.3474, -0.3121,  0.3060, -0.3228,
            0.3123, -0.1322, -0.1340,  0.2122,  0.4841, -0.1493, -0.2204,
            0.1846,  0.1844,  0.1363,  0.2888, -0.1850, -0.3460,  0.1411,
            0.3320, -0.0729, -0.2061,  0.3027,  0.1849,  0.1027,  0.0321,
            0.1789,  0.4468, -0.1360,  0.1420,  0.0072,  0.1781, -0.1733,
           -0.2032, -0.1516,  0.2629, -0.0523, -0.2634,  0.1443, -0.0333,
           -0.0531,  0.2466,  0.0352,  0.1251,  0.0216, -0.1368, -0.1189,
            0.1208, -0.2075,  0.3448,  0.0245, -0.2518, -0.4847, -0.2586,
           -0.4369,  0.0239,  0.2281,  0.1734, -0.0639, -0.1731, -0.1394,
            0.1520, -0.0963,  0.1410, -0.0228, -0.1736,  0.1204, -0.1059,
            0.1951,  0.2924, -0.2550,  0.1103, -0.0120, -0.1580, -0.0595,
           -

In [122]:
encoder_hidden

tensor([[[-0.3660, -0.1374, -0.1627,  0.1791,  0.1956,  0.0855, -0.2766,
           0.0155, -0.1747,  0.1660, -0.0397, -0.0998,  0.1683,  0.0571,
           0.4571, -0.1024,  0.1151, -0.3474, -0.3121,  0.3060, -0.3228,
           0.3123, -0.1322, -0.1340,  0.2122,  0.4841, -0.1493, -0.2204,
           0.1846,  0.1844,  0.1363,  0.2888, -0.1850, -0.3460,  0.1411,
           0.3320, -0.0729, -0.2061,  0.3027,  0.1849,  0.1027,  0.0321,
           0.1789,  0.4468, -0.1360,  0.1420,  0.0072,  0.1781, -0.1733,
          -0.2032, -0.1516,  0.2629, -0.0523, -0.2634,  0.1443, -0.0333,
          -0.0531,  0.2466,  0.0352,  0.1251,  0.0216, -0.1368, -0.1189,
           0.1208, -0.2075,  0.3448,  0.0245, -0.2518, -0.4847, -0.2586,
          -0.4369,  0.0239,  0.2281,  0.1734, -0.0639, -0.1731, -0.1394,
           0.1520, -0.0963,  0.1410, -0.0228, -0.1736,  0.1204, -0.1059,
           0.1951,  0.2924, -0.2550,  0.1103, -0.0120, -0.1580, -0.0595,
          -0.1540,  0.1275,  0.2796, -0.2650, -0.19

In [123]:
for ei in range(1, input_length):
    encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

In [124]:
encoder_output, encoder_hidden

(tensor([[[ 0.5003,  0.0664,  0.0607, -0.0306,  0.3171, -0.1654, -0.0281,
           -0.4843, -0.3115,  0.4210, -0.2452,  0.2667, -0.3991,  0.5182,
            0.0693, -0.1961, -0.2001, -0.2919, -0.1187, -0.0310,  0.1213,
           -0.0053, -0.4287,  0.1027, -0.1622, -0.1453,  0.1038, -0.3443,
            0.0009,  0.1491, -0.1768, -0.6122, -0.4340, -0.4871, -0.1614,
           -0.1898, -0.0332,  0.0071, -0.1185, -0.3717,  0.0487, -0.0689,
            0.4097,  0.2257, -0.6109,  0.4033, -0.4832,  0.2019,  0.2767,
           -0.2665, -0.1330,  0.1546, -0.1259, -0.3879, -0.1704,  0.1605,
            0.2452,  0.1071,  0.0084,  0.0010, -0.1363,  0.5580, -0.1690,
            0.3365,  0.3151, -0.2265, -0.1725, -0.2060,  0.4324,  0.1258,
            0.0176,  0.2217,  0.2560, -0.1881,  0.1230, -0.1382,  0.0820,
           -0.4468, -0.3240,  0.2064, -0.0768,  0.0479, -0.2700,  0.0912,
            0.3942, -0.2609, -0.2430,  0.3965,  0.3604, -0.0798, -0.0924,
            0.2362, -0.1323, -0.5466, 

In [125]:
decoder_input = torch.tensor([[SOS_token]], device=device)

In [126]:
decoder_input

tensor([[0]], device='cuda:0')

In [129]:
decoder_hidden = encoder_hidden
decoder_attentions = torch.zeros(MAX_LENGTH, MAX_LENGTH)

In [132]:
decoder_attentions

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(
            decoder_input, decoder_hidden)
        if use_teacher_forcing: # Teacher forcing: Feed the target as the next decoder input
            ### BEGIN SOLUTION
            decoder_input = target_tensor[di]  # Teacher forcing
            ### END SOLUTION
        else:                   # Without teacher forcing: use its own predictions as the next input
            ### BEGIN SOLUTION
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as decoder input
            ### END SOLUTION
        loss += criterion(decoder_output, target_tensor[di])
        # Stop if terminate token (EOS) is generated
        ### BEGIN SOLUTION
        if decoder_input.item() == EOS_token: # Terminate token is returned. Stop
            break
        ### END SOLUTION

    # Perform gradient step
    ### BEGIN SOLUTION
    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()
    ### END SOLUTION

    return loss.item() / target_length

In [137]:
use_teacher_forcing = True if random.random() < 0.5 else False
use_teacher_forcing

False

In [141]:
#decoder_output not defined yet
#decoder_hidden (era l'output dell'encoder)
#decoder_input # tensor([[0]], device='cuda:0')
decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)

tensor([[0]], device='cuda:0')

In [143]:
decoder_output, decoder_hidden

(tensor([[-8.0365, -8.0645, -7.8952,  ..., -7.9550, -8.0629, -8.0634]],
        device='cuda:0', grad_fn=<LogSoftmaxBackward0>),
 tensor([[[ 0.2803, -0.1187,  0.1272, -0.1398,  0.3039,  0.1148, -0.1754,
           -0.2082,  0.1313,  0.0236, -0.3504, -0.0618, -0.2385,  0.1751,
            0.2292,  0.0911,  0.0673, -0.1782, -0.0619, -0.1751,  0.1129,
            0.3809, -0.4244,  0.1414, -0.1107, -0.1238, -0.0176, -0.3950,
            0.1365,  0.2115, -0.0188, -0.2534, -0.1923,  0.1726,  0.3072,
           -0.0378, -0.2927, -0.0770, -0.2897, -0.1238,  0.2608, -0.4358,
            0.2715, -0.0042,  0.1687,  0.2844, -0.1135, -0.0777,  0.2595,
            0.0401, -0.1007,  0.1959, -0.1336,  0.0626,  0.0834, -0.0325,
            0.1240, -0.1252, -0.0791,  0.1350,  0.2160,  0.3585, -0.0300,
            0.2731, -0.0146, -0.2522,  0.1297, -0.0670,  0.1792,  0.1342,
            0.1316,  0.0170,  0.1274, -0.3153, -0.2306,  0.1310,  0.1500,
           -0.5460, -0.2305, -0.0824, -0.0102, -0.2978,  

In [145]:
decoder_output.shape, decoder_hidden.shape

(torch.Size([1, 2803]), torch.Size([1, 1, 256]))

In [148]:
target_tensor = output_tensor
target_tensor[0]

tensor([2], device='cuda:0')

In [149]:
criterion = nn.NLLLoss()
criterion(decoder_output, target_tensor[0])

tensor(7.8952, device='cuda:0', grad_fn=<NllLossBackward0>)

In [151]:
decoder_output, target_tensor[0]

(tensor([[-8.0365, -8.0645, -7.8952,  ..., -7.9550, -8.0629, -8.0634]],
        device='cuda:0', grad_fn=<LogSoftmaxBackward0>),
 tensor([2], device='cuda:0'))

In [152]:
decoder_input.item()

0

In [73]:
encoder.embedding, encoder.gru

(Embedding(4345, 256), GRU(256, 256))

In [161]:
trainIters(encoder, decoder, 75000, print_every=5000)

1m 6s (- 15m 28s) (5000 6%) 2.9076
2m 7s (- 13m 48s) (10000 13%) 2.3489
3m 8s (- 12m 32s) (15000 20%) 2.0182
4m 8s (- 11m 24s) (20000 26%) 1.7885
5m 11s (- 10m 22s) (25000 33%) 1.5558
6m 11s (- 9m 17s) (30000 40%) 1.3790
7m 12s (- 8m 14s) (35000 46%) 1.2102
8m 12s (- 7m 11s) (40000 53%) 1.0689
9m 12s (- 6m 8s) (45000 60%) 0.9581
10m 14s (- 5m 7s) (50000 66%) 0.8400
11m 14s (- 4m 5s) (55000 73%) 0.7675
12m 14s (- 3m 3s) (60000 80%) 0.6827
13m 13s (- 2m 2s) (65000 86%) 0.6066
14m 13s (- 1m 0s) (70000 93%) 0.5455
15m 13s (- 0m 0s) (75000 100%) 0.4780


In [163]:
evaluateRandomly(encoder, decoder)

> il sait nager comme un poisson .
= he is able to swim like a fish .
< he is able to swim a fish . <EOS>

> ce sont des employees a temps partiel .
= they re part time employees .
< they re part time time . <EOS>

> c est un mordu de cinema .
= he s a movie buff .
< he s a movie . <EOS>

> je peux nager dans la riviere .
= i am allowed to swim in the river .
< i am allowed to swim in the river . <EOS>

> ils ne vont pas revenir .
= they re not coming back .
< they re not coming back . <EOS>

> c est mon patron .
= he is my boss .
< he is my boss . <EOS>

> il est allergique aux chats .
= he s allergic to cats .
< he s allergic to cats . <EOS>

> je suis tres occupee tom .
= i m really busy tom .
< i m very busy tom . <EOS>

> vous etes suffisante .
= you re conceited .
< you re conceited . <EOS>

> je suis assez en colere .
= i m pretty angry .
< i m very angry . <EOS>



In [165]:
evaluate(encoder, decoder, "je ai tres faim")


['i', 'm', 'very', 'hungry', '.', '<EOS>']

In [166]:
evaluate(encoder, decoder, "maintenant je dois preparer le dejeuner")

['i', 'm', 'having', 'the', 'same', 'of', '.', '.', '<EOS>']

In [174]:
evaluate(encoder, decoder, "je suis tres bon")

['i', 'm', 'very', 'good', '.', '<EOS>']

In [169]:
output_lang.word2count["lunch"]

7

In [176]:
### BEGIN HIDDEN TESTS
res = []
for pair in pairs[:40]:
    output_words = evaluate(encoder, decoder, pair[0])
    res.append(output_words == pair[1].split(' ') + ['<EOS>'])
assert np.mean(res) > 0.55
### END HIDDEN TESTS

AssertionError: 